In [1]:
from rastrigin import *

In [ ]:
n_exec = 100

In [ ]:
fitness = list()

for i in range(n_exec):
    
    GA = GeneticAlgorithm(n = 10, m = 10, l = 20, max_eval = 10000, mut_prob = 0.9, cross_prob = 0.9, k = 5)
    GA.solve()
    fitness.append(GA.best_fitness)

Generations:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
print(min(fitness))
print(max(fitness))
print(sum(fitness)/len(fitness))